In [9]:
import requests
from bs4 import BeautifulSoup

def fetch_and_modify_html(url, output_file):
    # 读取网页内容
    response = requests.get(url)
    
    if response.status_code == 200:
        html_content = response.text
        
        # 解析 HTML 内容
        soup = BeautifulSoup(html_content, 'html.parser')

        # 查找所有包含 `time-slot fulled`、`time-slot booked` 或 `time-slot closed` 类的 `span` 标签
        for span in soup.find_all('span', class_=['time-slot fulled', 'time-slot booked']):
            span['onclick'] = "toggleSlot(this)"
        
        # 添加 JavaScript 代码
        script = soup.new_tag('script')
        script.string = '''
        function toggleSlot(span) {
            // 获取日期和时间段文本，作为唯一标识符
            const dateInfo = span.closest('.slot').querySelector('.date-info').textContent.trim();
            const slotTime = span.textContent.trim();
            const slotId = `${dateInfo}_${slotTime}`; // 生成唯一标识符

            // 切换状态并存储
            if (span.classList.contains('fulled')) {
                span.classList.remove('fulled');
                span.classList.add('booked');
                localStorage.setItem(slotId, 'booked');
            } else if (span.classList.contains('booked')) {
                span.classList.remove('booked');
                span.classList.add('fulled');
                localStorage.setItem(slotId, 'fulled');
            }
        }

        document.addEventListener('DOMContentLoaded', () => {
            const slots = document.querySelectorAll('.time-slot');
            slots.forEach(slot => {
                const dateInfo = slot.closest('.slot').querySelector('.date-info').textContent.trim();
                const slotTime = slot.textContent.trim();
                const slotId = `${dateInfo}_${slotTime}`; // 生成唯一标识符

                // 从 localStorage 中恢复状态
                const savedStatus = localStorage.getItem(slotId);
                if (savedStatus) {
                    slot.classList.remove('fulled', 'booked', 'closed');
                    slot.classList.add(savedStatus);
                }

                slot.addEventListener('click', () => {
                    toggleSlot(slot);
                });
            });
        });
        '''
        soup.body.append(script)  # 将脚本添加到 `body` 的末尾

        # 手动替换 &nbsp; 只在指定元素中
        for green_slot in soup.find_all('span', class_='green_slot'):
            if green_slot.string:
                green_slot.string.replace_with(green_slot.string.replace(" ", "\xa0"))

        # 保存修改后的 HTML 文件
        with open(f'templates/{output_file}', 'w', encoding='utf-8') as file:
            file.write(str(soup))

In [10]:
fetch_and_modify_html("https://cgyy.xmu.edu.cn/room/2", "room/2.html")

In [9]:
import requests
from bs4 import BeautifulSoup

# 2. 从网页端获取 <ul class="slot-list slot-list-2"> 的内容
url = "https://cgyy.xmu.edu.cn/room/2"  # 替换为实际网址
response = requests.get(url)
web_soup = BeautifulSoup(response.text, 'html.parser')
web_ul_element = web_soup.find('ul', class_='slot-list slot-list-2')

# 检查是否成功找到元素
if not web_ul_element:
    print("未找到网页中的 <ul class='slot-list slot-list-2'> 元素")
else:
    # print(web_ul_element.prettify())
    # 2. 读取本地 HTML 文件
    with open("templates/room/2.html", "r", encoding="utf-8") as file:  # 替换为你的本地文件名
        local_soup = BeautifulSoup(file, 'html.parser')
    
    # 3. 替换本地文件中的 <ul class="slot-list slot-list-2"> 的内容
    local_ul_element = local_soup.find('ul', class_='slot-list slot-list-2')
    
    if local_ul_element:
        # print(local_ul_element.prettify())
        # 用网页端的内容替换本地的 <ul> 元素内容
        local_ul_element.replace_with(web_ul_element)
        
        # 4. 将修改后的 HTML 保存回本地文件
        with open("templates/room/2.html", "w", encoding="utf-8") as file:
            file.write(str(local_soup))
        
        print("替换完成并保存到本地文件")
    else:
        print("未找到本地文件中的 <ul class='slot-list slot-list-2'> 元素")

替换完成并保存到本地文件
